# Teradata DBQL Summary Table History Report

This notebook demonstrates how to retrieve and analyze DBQL Summary Table History from Teradata PDCR data using the `PDCRInfoReport` class.

**Report Parameters:**
- User filter: `%` (all users)
- Time range: Last 30 days
- Data source: `PDCRINFO.DBQLSummaryTbl_Hst`

**Analysis Includes:**
- Top CPU consumers
- Query performance trends
- Daily workload patterns
- User activity analysis

## 1. Import Required Libraries

Import necessary libraries for PDCR reporting and data analysis.

In [ ]:
import logging
import sys
from pathlib import Path
from datetime import date, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Add src to path for imports
sys.path.insert(0, str(Path.cwd()))

# Import the reporting module
from src.reports import PDCRInfoReport
from src.connection import TeradataConnectionError

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

## 2. Configure Date Range

Calculate the date range for the last 30 days of data.

In [ ]:
# Calculate last 30 days date range
end_date = date.today() - timedelta(days=1)  # Yesterday
start_date = end_date - timedelta(days=30)   # 30 days ago

# User filter pattern
user_pattern = "%"  # All users

print(f"Date Range:")
print(f"  Start Date: {start_date}")
print(f"  End Date:   {end_date}")
print(f"  User Pattern: {user_pattern}")
print(f"  Days: {(end_date - start_date).days + 1}")

## 3. Initialize PDCR Report Generator

Create an instance of the `PDCRInfoReport` class to access PDCR data.

In [ ]:
try:
    # Initialize the report generator
    report = PDCRInfoReport()
    print("✓ PDCRInfoReport initialized successfully")
    
    # List available environments
    environments = report.conn_mgr.list_environments()
    print(f"✓ Available environments: {environments}")
    
except TeradataConnectionError as e:
    print(f"✗ Connection Error: {e}")
    print("\nPlease ensure:")
    print("1. td_env.yaml file exists in the project root")
    print("2. Copy td_env.yaml.template to td_env.yaml")
    print("3. Update credentials for your test/prod environments")

## 4. Retrieve DBQL Summary Table History Data

Query `PDCRINFO.DBQLSummaryTbl_Hst` for query performance metrics over the last 30 days.

In [ ]:
try:
    # Retrieve DBQL summary history
    df = report.get_DBQLSummaryTable_History(
        env_name='test',  # Change to 'prod' for production data
        start_date=start_date,
        end_date=end_date,
        user_name=user_pattern
    )
    
    print(f"✓ Retrieved {len(df):,} rows from PDCRINFO.DBQLSummaryTbl_Hst")
    print(f"\nDataFrame Shape: {df.shape}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
except Exception as e:
    print(f"✗ Error retrieving DBQL summary data: {e}")
    df = None

## 5. Display Sample Data

Preview the first few rows to understand the data structure.

In [ ]:
if df is not None and not df.empty:
    print("First 10 rows:")
    display(df.head(10))
    
    print("\nColumn Data Types:")
    print(df.dtypes)
else:
    print("No data available to display.")

## 6. Data Summary Statistics

Analyze the query performance metrics across all retrieved data.

In [ ]:
if df is not None and not df.empty:
    print("=" * 80)
    print("DBQL SUMMARY STATISTICS")
    print("=" * 80)
    
    # Date range
    print(f"\nDate Range:")
    print(f"  First Log Date: {df['LogDate'].min()}")
    print(f"  Last Log Date:  {df['LogDate'].max()}")
    print(f"  Unique Dates:   {df['LogDate'].nunique()}")
    
    # User coverage
    print(f"\nUser Coverage:")
    print(f"  Unique Users:     {df['UserName'].nunique()}")
    print(f"  Total Records:    {len(df):,}")
    print(f"  Total Queries:    {df['QueryCount'].sum():,}")
    
    # CPU usage statistics (in seconds, convert to hours)
    print(f"\nCPU Usage (Hours):")
    print(f"  Total AMP CPU:      {df['AMPCPUTime'].sum() / 3600:,.2f}")
    print(f"  Total Parser CPU:   {df['ParserCPUTime'].sum() / 3600:,.2f}")
    print(f"  Total Combined CPU: {(df['AMPCPUTime'].sum() + df['ParserCPUTime'].sum()) / 3600:,.2f}")
    print(f"  Avg CPU per Query:  {((df['AMPCPUTime'].sum() + df['ParserCPUTime'].sum()) / df['QueryCount'].sum()):.4f} seconds")
    
    # Query execution time
    print(f"\nQuery Execution Time:")
    print(f"  Total Query Seconds: {df['QuerySeconds'].sum():,.2f}")
    print(f"  Avg Query Duration:  {(df['QuerySeconds'].sum() / df['QueryCount'].sum()):.4f} seconds")
    
    # I/O statistics
    print(f"\nI/O Statistics:")
    print(f"  Total IO Count:     {df['TotalIOCount'].sum():,.0f}")
    print(f"  Avg IO per Query:   {(df['TotalIOCount'].sum() / df['QueryCount'].sum()):,.0f}")
    if 'TotalIOInKB' in df.columns:
        print(f"  Total IO (GB):      {df['TotalIOInKB'].sum() / 1024**2:,.2f}")
    
else:
    print("No data available.")

## 7. Top 10 CPU Users

Identify the top 10 users by total CPU time (AMP + Parser).

In [ ]:
if df is not None and not df.empty:
    # Calculate total CPU by user
    user_cpu = df.groupby('UserName').agg({
        'AMPCPUTime': 'sum',
        'ParserCPUTime': 'sum',
        'QueryCount': 'sum',
        'QuerySeconds': 'sum',
        'TotalIOCount': 'sum'
    }).reset_index()
    
    # Add total CPU column
    user_cpu['TotalCPU'] = user_cpu['AMPCPUTime'] + user_cpu['ParserCPUTime']
    user_cpu['TotalCPU_Hours'] = user_cpu['TotalCPU'] / 3600
    
    # Get top 10
    top_10_cpu = user_cpu.nlargest(10, 'TotalCPU')
    
    print("\n" + "=" * 80)
    print("TOP 10 CPU USERS")
    print("=" * 80)
    
    display(top_10_cpu[['UserName', 'QueryCount', 'TotalCPU_Hours', 'QuerySeconds', 'TotalIOCount']].style.format({
        'QueryCount': '{:,.0f}',
        'TotalCPU_Hours': '{:.2f}',
        'QuerySeconds': '{:,.2f}',
        'TotalIOCount': '{:,.0f}'
    }))
else:
    print("No data available.")

## 8. Top 10 Query Count Users

Identify the most active users by number of queries executed.

In [ ]:
if df is not None and not df.empty:
    # Get top 10 by query count
    top_10_queries = user_cpu.nlargest(10, 'QueryCount')
    
    print("\n" + "=" * 80)
    print("TOP 10 MOST ACTIVE USERS (by Query Count)")
    print("=" * 80)
    
    display(top_10_queries[['UserName', 'QueryCount', 'TotalCPU_Hours', 'QuerySeconds', 'TotalIOCount']].style.format({
        'QueryCount': '{:,.0f}',
        'TotalCPU_Hours': '{:.2f}',
        'QuerySeconds': '{:,.2f}',
        'TotalIOCount': '{:,.0f}'
    }))
else:
    print("No data available.")

## 9. Visualize Top 10 CPU Users

Create a horizontal bar chart showing CPU consumption breakdown.

In [ ]:
if df is not None and not df.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Prepare data
    top_10_sorted = top_10_cpu.sort_values('TotalCPU_Hours')
    
    # Create stacked horizontal bars
    y_pos = np.arange(len(top_10_sorted))
    amp_hours = top_10_sorted['AMPCPUTime'] / 3600
    parser_hours = top_10_sorted['ParserCPUTime'] / 3600
    
    ax.barh(y_pos, amp_hours, label='AMP CPU Time', color='#2E86AB')
    ax.barh(y_pos, parser_hours, left=amp_hours, label='Parser CPU Time', color='#A23B72')
    
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top_10_sorted['UserName'])
    ax.set_xlabel('CPU Time (Hours)', fontweight='bold')
    ax.set_ylabel('User Name', fontweight='bold')
    ax.set_title('Top 10 CPU Users - Last 30 Days', fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='lower right')
    ax.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization.")

## 10. Daily CPU Usage Trend

Plot total CPU usage per day to identify workload patterns.

In [ ]:
if df is not None and not df.empty:
    # Aggregate by date
    daily_cpu = df.groupby('LogDate').agg({
        'AMPCPUTime': 'sum',
        'ParserCPUTime': 'sum',
        'QueryCount': 'sum',
        'TotalIOCount': 'sum'
    }).reset_index()
    
    daily_cpu['TotalCPU_Hours'] = (daily_cpu['AMPCPUTime'] + daily_cpu['ParserCPUTime']) / 3600
    daily_cpu['AMPCPUTime_Hours'] = daily_cpu['AMPCPUTime'] / 3600
    daily_cpu['ParserCPUTime_Hours'] = daily_cpu['ParserCPUTime'] / 3600
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Plot 1: Stacked area chart
    ax1.fill_between(daily_cpu['LogDate'], 0, daily_cpu['AMPCPUTime_Hours'], 
                     alpha=0.7, label='AMP CPU', color='#2E86AB')
    ax1.fill_between(daily_cpu['LogDate'], daily_cpu['AMPCPUTime_Hours'], 
                     daily_cpu['TotalCPU_Hours'], alpha=0.7, label='Parser CPU', color='#A23B72')
    
    ax1.set_ylabel('CPU Time (Hours)', fontweight='bold')
    ax1.set_title('Daily CPU Usage - Stacked View', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Plot 2: Line chart with trend
    ax2.plot(daily_cpu['LogDate'], daily_cpu['TotalCPU_Hours'], 
            marker='o', linewidth=2, markersize=5, label='Total CPU', color='#F18F01')
    
    # Add trend line
    if len(daily_cpu) > 1:
        x_numeric = np.arange(len(daily_cpu))
        z = np.polyfit(x_numeric, daily_cpu['TotalCPU_Hours'], 1)
        p = np.poly1d(z)
        ax2.plot(daily_cpu['LogDate'], p(x_numeric), "--", alpha=0.7, linewidth=2, 
                label='Trend', color='#C73E1D')
    
    ax2.set_xlabel('Date', fontweight='bold')
    ax2.set_ylabel('Total CPU Time (Hours)', fontweight='bold')
    ax2.set_title('Daily Total CPU Usage with Trend', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print(f"\nDaily CPU Statistics:")
    print(f"  Average: {daily_cpu['TotalCPU_Hours'].mean():.2f} hours")
    print(f"  Peak:    {daily_cpu['TotalCPU_Hours'].max():.2f} hours on {daily_cpu.loc[daily_cpu['TotalCPU_Hours'].idxmax(), 'LogDate']}")
    print(f"  Minimum: {daily_cpu['TotalCPU_Hours'].min():.2f} hours on {daily_cpu.loc[daily_cpu['TotalCPU_Hours'].idxmin(), 'LogDate']}")
else:
    print("No data available.")

## 11. Daily Query Count Trend

Analyze the number of queries executed per day.

In [ ]:
if df is not None and not df.empty:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Bar chart of daily query counts
    ax.bar(daily_cpu['LogDate'], daily_cpu['QueryCount'], color='#2E86AB', alpha=0.7, edgecolor='black')
    
    # Add average line
    avg_queries = daily_cpu['QueryCount'].mean()
    ax.axhline(y=avg_queries, color='red', linestyle='--', linewidth=2, 
               label=f'Average: {avg_queries:,.0f} queries/day')
    
    ax.set_xlabel('Date', fontweight='bold')
    ax.set_ylabel('Query Count', fontweight='bold')
    ax.set_title('Daily Query Volume', fontsize=14, fontweight='bold', pad=20)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    # Format y-axis with thousands separator
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x):,}'))
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available.")

## 12. CPU Usage by Day of Week

Identify weekly patterns in CPU consumption.

In [ ]:
if df is not None and not df.empty:
    # Add day of week
    df_copy = df.copy()
    df_copy['DayOfWeek'] = pd.to_datetime(df_copy['LogDate']).dt.day_name()
    df_copy['TotalCPU'] = df_copy['AMPCPUTime'] + df_copy['ParserCPUTime']
    
    # Aggregate by day of week
    dow_cpu = df_copy.groupby('DayOfWeek').agg({
        'TotalCPU': 'sum',
        'QueryCount': 'sum'
    }).reset_index()
    
    # Sort by day of week
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    dow_cpu['DayOfWeek'] = pd.Categorical(dow_cpu['DayOfWeek'], categories=day_order, ordered=True)
    dow_cpu = dow_cpu.sort_values('DayOfWeek')
    dow_cpu['TotalCPU_Hours'] = dow_cpu['TotalCPU'] / 3600
    
    # Create plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(dow_cpu)))
    bars = ax.bar(dow_cpu['DayOfWeek'], dow_cpu['TotalCPU_Hours'], color=colors, edgecolor='black')
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}h',
                ha='center', va='bottom', fontweight='bold')
    
    ax.set_xlabel('Day of Week', fontweight='bold')
    ax.set_ylabel('Total CPU Time (Hours)', fontweight='bold')
    ax.set_title('CPU Usage by Day of Week', fontsize=14, fontweight='bold', pad=20)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available.")

## 13. Top 5 Users - Daily CPU Trends

Compare CPU usage trends for the top 5 users over time.

In [ ]:
if df is not None and not df.empty:
    # Get top 5 users
    top_5_users = top_10_cpu.head(5)['UserName'].tolist()
    
    # Filter and aggregate
    df_top5 = df[df['UserName'].isin(top_5_users)].copy()
    df_top5['TotalCPU'] = df_top5['AMPCPUTime'] + df_top5['ParserCPUTime']
    
    user_daily = df_top5.groupby(['LogDate', 'UserName'])['TotalCPU'].sum().reset_index()
    user_daily['TotalCPU_Hours'] = user_daily['TotalCPU'] / 3600
    
    # Create plot
    fig, ax = plt.subplots(figsize=(14, 7))
    
    for user in top_5_users:
        user_data = user_daily[user_daily['UserName'] == user]
        ax.plot(user_data['LogDate'], user_data['TotalCPU_Hours'], 
               marker='o', linewidth=2, markersize=4, label=user)
    
    ax.set_xlabel('Date', fontweight='bold')
    ax.set_ylabel('CPU Time (Hours)', fontweight='bold')
    ax.set_title('Top 5 Users - Daily CPU Usage Trends', fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', frameon=True, shadow=True)
    ax.grid(alpha=0.3)
    ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available.")

## 14. Query Performance Heatmap

Create a heatmap showing average CPU time per query for top users by day.

In [ ]:
if df is not None and not df.empty:
    # Get top 10 users
    top_users = top_10_cpu.head(10)['UserName'].tolist()
    
    # Filter and calculate metrics
    df_top = df[df['UserName'].isin(top_users)].copy()
    df_top['TotalCPU'] = df_top['AMPCPUTime'] + df_top['ParserCPUTime']
    df_top['AvgCPUPerQuery'] = df_top['TotalCPU'] / df_top['QueryCount']
    
    # Create pivot table
    pivot_data = df_top.pivot_table(
        values='AvgCPUPerQuery',
        index='UserName',
        columns='LogDate',
        aggfunc='mean'
    )
    
    # Create heatmap
    fig, ax = plt.subplots(figsize=(16, 8))
    
    sns.heatmap(pivot_data, annot=False, fmt='.2f', cmap='YlOrRd', 
                cbar_kws={'label': 'Avg CPU per Query (seconds)'}, ax=ax)
    
    ax.set_xlabel('Date', fontweight='bold')
    ax.set_ylabel('User Name', fontweight='bold')
    ax.set_title('Average CPU Time per Query - Heatmap', fontsize=14, fontweight='bold', pad=20)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available.")

## 15. I/O vs CPU Scatter Plot

Analyze the relationship between I/O operations and CPU time for top users.

In [ ]:
if df is not None and not df.empty:
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Use top 10 user data
    for i, user in enumerate(top_10_cpu['UserName'].tolist()):
        user_data = top_10_cpu[top_10_cpu['UserName'] == user].iloc[0]
        ax.scatter(user_data['TotalIOCount'], user_data['TotalCPU_Hours'], 
                  s=user_data['QueryCount']/100, alpha=0.6, label=user)
    
    ax.set_xlabel('Total I/O Count', fontweight='bold')
    ax.set_ylabel('Total CPU Time (Hours)', fontweight='bold')
    ax.set_title('I/O vs CPU Usage (bubble size = query count)', 
                fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', frameon=True, shadow=True, ncol=2)
    ax.grid(alpha=0.3)
    
    # Format axes
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1e6)}M'))
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available.")

## 16. Query Efficiency Analysis

Calculate and visualize query efficiency metrics.

In [ ]:
if df is not None and not df.empty:
    # Calculate efficiency metrics for top 10 users
    efficiency = top_10_cpu.copy()
    efficiency['AvgCPUPerQuery'] = efficiency['TotalCPU'] / efficiency['QueryCount']
    efficiency['AvgIOPerQuery'] = efficiency['TotalIOCount'] / efficiency['QueryCount']
    efficiency['AvgDurationPerQuery'] = efficiency['QuerySeconds'] / efficiency['QueryCount']
    
    print("\n" + "=" * 80)
    print("QUERY EFFICIENCY METRICS - TOP 10 CPU USERS")
    print("=" * 80)
    
    display(efficiency[['UserName', 'QueryCount', 'AvgCPUPerQuery', 
                        'AvgDurationPerQuery', 'AvgIOPerQuery']].style.format({
        'QueryCount': '{:,.0f}',
        'AvgCPUPerQuery': '{:.4f}',
        'AvgDurationPerQuery': '{:.4f}',
        'AvgIOPerQuery': '{:,.0f}'
    }))
    
    # Visualize efficiency
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Plot 1: Average CPU per query
    efficiency_sorted = efficiency.sort_values('AvgCPUPerQuery', ascending=True)
    ax1.barh(efficiency_sorted['UserName'], efficiency_sorted['AvgCPUPerQuery'], 
            color='#2E86AB', edgecolor='black')
    ax1.set_xlabel('Avg CPU per Query (seconds)', fontweight='bold')
    ax1.set_ylabel('User Name', fontweight='bold')
    ax1.set_title('Average CPU Time per Query', fontsize=12, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    
    # Plot 2: Average Duration per query
    duration_sorted = efficiency.sort_values('AvgDurationPerQuery', ascending=True)
    ax2.barh(duration_sorted['UserName'], duration_sorted['AvgDurationPerQuery'], 
            color='#A23B72', edgecolor='black')
    ax2.set_xlabel('Avg Duration per Query (seconds)', fontweight='bold')
    ax2.set_ylabel('User Name', fontweight='bold')
    ax2.set_title('Average Query Duration', fontsize=12, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available.")

## 17. Export Results to CSV

Save the results to CSV files for further analysis or reporting.

In [ ]:
if df is not None and not df.empty:
    # Create output directory if it doesn't exist
    output_dir = Path('output')
    output_dir.mkdir(exist_ok=True)
    
    # Export full dataset
    filename = f"dbql_summary_history_{start_date}_{end_date}.csv"
    output_path = output_dir / filename
    df.to_csv(output_path, index=False)
    print(f"✓ Full dataset exported to: {output_path}")
    print(f"  Rows: {len(df):,}")
    print(f"  File size: {output_path.stat().st_size / 1024**2:.2f} MB")
    
    # Export top 10 CPU users
    summary_file = f"dbql_top10_cpu_users_{start_date}_{end_date}.csv"
    summary_path = output_dir / summary_file
    top_10_cpu.to_csv(summary_path, index=False)
    print(f"\n✓ Top 10 CPU users exported to: {summary_path}")
    
    # Export daily summary
    daily_file = f"dbql_daily_summary_{start_date}_{end_date}.csv"
    daily_path = output_dir / daily_file
    daily_cpu.to_csv(daily_path, index=False)
    print(f"\n✓ Daily summary exported to: {daily_path}")
else:
    print("No data to export.")

## 18. Close Connections

Properly clean up database connections when done.

In [ ]:
# Close all connections
report.close()
print("✓ Database connections closed successfully!")

# Teradata DBQL Summary Table History Report

This notebook demonstrates how to retrieve and analyze DBQL Summary Table History from Teradata PDCR data using the `PDCRInfoReport` class.

**Report Parameters:**
- User filter: `%` (all users)
- Time range: Last 30 days
- Data source: `PDCRINFO.DBQLSummaryTbl_Hst`

**Analysis Includes:**
- Top 10 CPU users
- CPU usage trends over time

## 1. Import Required Libraries

Import necessary libraries for PDCR reporting and data analysis.

In [ ]:
import logging
import sys
from pathlib import Path
from datetime import date, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Add src to path for imports
sys.path.insert(0, str(Path.cwd()))

# Import the reporting module
from src.reports import PDCRInfoReport
from src.connection import TeradataConnectionError

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✓ Libraries imported successfully!")

## 2. Configure Date Range

Calculate the date range for the last 30 days of data.

In [ ]:
# Calculate date range - last 30 days
end_date = date.today() - timedelta(days=1)  # Yesterday
start_date = end_date - timedelta(days=30)  # 30 days ago

# User filter
user_name = '%'  # All users

print("Date Range:")
print(f"  Start Date: {start_date}")
print(f"  End Date:   {end_date}")
print(f"  User Pattern: {user_name}")
print(f"  Days: {(end_date - start_date).days}")

## 3. Initialize Report Generator

Create an instance of PDCRInfoReport using the default configuration.

In [ ]:
# Initialize the report generator
report = PDCRInfoReport()
print("✓ PDCRInfoReport initialized successfully!")

## 4. Fetch DBQL Summary Data

Retrieve DBQL Summary Table History data from the Teradata environment.

In [ ]:
# Fetch the DBQL summary data
try:
    df = report.get_DBQLSummaryTable_History(
        env_name='prod',
        start_date=start_date,
        end_date=end_date,
        user_name=user_name
    )
    print(f"✓ Successfully retrieved {len(df):,} records")
    print(f"\nDate range in data: {df['LogDate'].min()} to {df['LogDate'].max()}")
    print(f"Unique users: {df['UserName'].nunique():,}")
    print(f"Total queries: {df['QueryCount'].sum():,}")
except TeradataConnectionError as e:
    print(f"✗ Failed to retrieve data: {e}")
    df = None
except Exception as e:
    print(f"✗ Unexpected error: {e}")
    df = None

## 5. Data Overview

Display basic statistics and structure of the retrieved data.

In [ ]:
if df is not None and not df.empty:
    print("Data Shape:", df.shape)
    print("\nColumn Names:")
    print(df.columns.tolist())
    print("\nFirst few rows:")
    display(df.head())
    print("\nData Types:")
    print(df.dtypes)
else:
    print("No data available to display.")

## 6. Top 10 CPU Users - By Total AMP CPU Time

Identify the top 10 users by total AMP CPU time over the reporting period.

In [ ]:
if df is not None and not df.empty:
    # Aggregate CPU time by user
    top_cpu_users = df.groupby('UserName').agg({
        'AMPCPUTime': 'sum',
        'ParserCPUTime': 'sum',
        'QueryCount': 'sum',
        'QuerySeconds': 'sum',
        'TotalIOCount': 'sum'
    }).reset_index()
    
    # Calculate total CPU time
    top_cpu_users['TotalCPUTime'] = top_cpu_users['AMPCPUTime'] + top_cpu_users['ParserCPUTime']
    
    # Sort by total CPU time and get top 10
    top_cpu_users = top_cpu_users.sort_values('TotalCPUTime', ascending=False).head(10)
    
    # Convert CPU time to hours for readability
    top_cpu_users['AMPCPUTime_Hours'] = top_cpu_users['AMPCPUTime'] / 3600
    top_cpu_users['ParserCPUTime_Hours'] = top_cpu_users['ParserCPUTime'] / 3600
    top_cpu_users['TotalCPUTime_Hours'] = top_cpu_users['TotalCPUTime'] / 3600
    
    print("\n=== TOP 10 CPU USERS ===")
    print("\nSorted by Total CPU Time (AMP + Parser)\n")
    
    display(top_cpu_users[[
        'UserName', 'QueryCount', 'TotalCPUTime_Hours', 
        'AMPCPUTime_Hours', 'ParserCPUTime_Hours', 'TotalIOCount'
    ]].style.format({
        'QueryCount': '{:,.0f}',
        'TotalCPUTime_Hours': '{:.2f}',
        'AMPCPUTime_Hours': '{:.2f}',
        'ParserCPUTime_Hours': '{:.2f}',
        'TotalIOCount': '{:,.0f}'
    }))
else:
    print("No data available for CPU user analysis.")

## 7. Visualize Top 10 CPU Users - Bar Chart

Create a bar chart showing the top 10 CPU users with AMP and Parser CPU breakdown.

In [ ]:
if df is not None and not df.empty:
    fig, ax = plt.subplots(figsize=(14, 8))
    
    x = np.arange(len(top_cpu_users))
    width = 0.35
    
    # Create bars
    bars1 = ax.bar(x - width/2, top_cpu_users['AMPCPUTime_Hours'], width, 
                   label='AMP CPU Time', color='#2E86AB')
    bars2 = ax.bar(x + width/2, top_cpu_users['ParserCPUTime_Hours'], width,
                   label='Parser CPU Time', color='#A23B72')
    
    # Customize chart
    ax.set_xlabel('User Name', fontsize=12, fontweight='bold')
    ax.set_ylabel('CPU Time (Hours)', fontsize=12, fontweight='bold')
    ax.set_title('Top 10 CPU Users - AMP vs Parser CPU Time', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(top_cpu_users['UserName'], rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization.")

## 8. CPU Usage Per Day - Overall System

Plot total CPU usage per day across all users to identify trends and patterns.

In [ ]:
if df is not None and not df.empty:
    # Aggregate CPU time by date
    daily_cpu = df.groupby('LogDate').agg({
        'AMPCPUTime': 'sum',
        'ParserCPUTime': 'sum',
        'QueryCount': 'sum',
        'TotalIOCount': 'sum'
    }).reset_index()
    
    # Calculate total CPU time in hours
    daily_cpu['TotalCPUTime_Hours'] = (daily_cpu['AMPCPUTime'] + daily_cpu['ParserCPUTime']) / 3600
    daily_cpu['AMPCPUTime_Hours'] = daily_cpu['AMPCPUTime'] / 3600
    daily_cpu['ParserCPUTime_Hours'] = daily_cpu['ParserCPUTime'] / 3600
    
    # Sort by date
    daily_cpu = daily_cpu.sort_values('LogDate')
    
    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))
    
    # Plot 1: Stacked area chart of CPU usage
    ax1.fill_between(daily_cpu['LogDate'], 0, daily_cpu['AMPCPUTime_Hours'], 
                     alpha=0.7, label='AMP CPU Time', color='#2E86AB')
    ax1.fill_between(daily_cpu['LogDate'], daily_cpu['AMPCPUTime_Hours'], 
                     daily_cpu['TotalCPUTime_Hours'],
                     alpha=0.7, label='Parser CPU Time', color='#A23B72')
    
    ax1.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax1.set_ylabel('CPU Time (Hours)', fontsize=12, fontweight='bold')
    ax1.set_title('Daily CPU Usage - Stacked View', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Plot 2: Line chart with trend
    ax2.plot(daily_cpu['LogDate'], daily_cpu['TotalCPUTime_Hours'], 
            marker='o', linewidth=2, label='Total CPU Time', color='#F18F01')
    
    # Add trend line
    if len(daily_cpu) > 1:
        x_numeric = np.arange(len(daily_cpu))
        z = np.polyfit(x_numeric, daily_cpu['TotalCPUTime_Hours'], 1)
        p = np.poly1d(z)
        ax2.plot(daily_cpu['LogDate'], p(x_numeric), "r--", 
                alpha=0.8, linewidth=2, label='Trend')
    
    ax2.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Total CPU Time (Hours)', fontsize=12, fontweight='bold')
    ax2.set_title('Daily Total CPU Usage with Trend', fontsize=14, fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n=== DAILY CPU USAGE STATISTICS ===")
    print(f"Average daily CPU time: {daily_cpu['TotalCPUTime_Hours'].mean():.2f} hours")
    print(f"Peak daily CPU time: {daily_cpu['TotalCPUTime_Hours'].max():.2f} hours on {daily_cpu.loc[daily_cpu['TotalCPUTime_Hours'].idxmax(), 'LogDate']}")
    print(f"Minimum daily CPU time: {daily_cpu['TotalCPUTime_Hours'].min():.2f} hours on {daily_cpu.loc[daily_cpu['TotalCPUTime_Hours'].idxmin(), 'LogDate']}")
    print(f"Total CPU time over period: {daily_cpu['TotalCPUTime_Hours'].sum():.2f} hours")
else:
    print("No data available for daily CPU visualization.")

## 9. CPU Usage Per Day - Top 10 Users

Plot CPU usage trends over time for each of the top 10 users.

In [ ]:
if df is not None and not df.empty:
    # Get top 10 user names from previous analysis
    top_10_users = top_cpu_users['UserName'].tolist()
    
    # Filter data for top 10 users
    df_top_users = df[df['UserName'].isin(top_10_users)].copy()
    
    # Aggregate by date and user
    user_daily_cpu = df_top_users.groupby(['LogDate', 'UserName']).agg({
        'AMPCPUTime': 'sum',
        'ParserCPUTime': 'sum'
    }).reset_index()
    
    # Calculate total CPU time in hours
    user_daily_cpu['TotalCPUTime_Hours'] = (user_daily_cpu['AMPCPUTime'] + user_daily_cpu['ParserCPUTime']) / 3600
    
    # Create subplots for top users
    fig, axes = plt.subplots(5, 2, figsize=(16, 20))
    axes = axes.flatten()
    
    for i, user in enumerate(top_10_users):
        ax = axes[i]
        user_data = user_daily_cpu[user_daily_cpu['UserName'] == user].sort_values('LogDate')
        
        if not user_data.empty:
            # Plot CPU usage
            ax.plot(user_data['LogDate'], user_data['TotalCPUTime_Hours'], 
                   marker='o', linewidth=2, label='CPU Time', color='#2E86AB')
            
            # Add trend line if enough data points
            if len(user_data) > 1:
                x_numeric = np.arange(len(user_data))
                z = np.polyfit(x_numeric, user_data['TotalCPUTime_Hours'], 1)
                p = np.poly1d(z)
                ax.plot(user_data['LogDate'], p(x_numeric), "r--", 
                       alpha=0.7, linewidth=2, label='Trend')
            
            ax.set_title(f'{user}', fontsize=11, fontweight='bold')
            ax.set_xlabel('Date', fontsize=9)
            ax.set_ylabel('CPU Time (Hours)', fontsize=9)
            ax.grid(alpha=0.3)
            ax.legend(fontsize=8)
            ax.tick_params(axis='x', rotation=45, labelsize=8)
            ax.tick_params(axis='y', labelsize=8)
    
    plt.suptitle('Daily CPU Usage - Top 10 Users', fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for user-level CPU visualization.")

## 10. Query Performance Metrics

Analyze average query performance metrics for top users.

In [ ]:
if df is not None and not df.empty:
    # Calculate average metrics per query for top users
    top_10_users = top_cpu_users['UserName'].tolist()
    df_top = df[df['UserName'].isin(top_10_users)].copy()
    
    performance_metrics = df_top.groupby('UserName').agg({
        'QueryCount': 'sum',
        'AMPCPUTime': 'sum',
        'ParserCPUTime': 'sum',
        'QuerySeconds': 'sum',
        'TotalIOCount': 'sum'
    }).reset_index()
    
    # Calculate averages per query
    performance_metrics['AvgCPUTimePerQuery'] = (performance_metrics['AMPCPUTime'] + performance_metrics['ParserCPUTime']) / performance_metrics['QueryCount']
    performance_metrics['AvgElapsedTimePerQuery'] = performance_metrics['QuerySeconds'] / performance_metrics['QueryCount']
    performance_metrics['AvgIOPerQuery'] = performance_metrics['TotalIOCount'] / performance_metrics['QueryCount']
    
    # Sort by average CPU time
    performance_metrics = performance_metrics.sort_values('AvgCPUTimePerQuery', ascending=False)
    
    print("\n=== QUERY PERFORMANCE METRICS - TOP 10 USERS ===")
    print("\nAverage resource consumption per query\n")
    
    display(performance_metrics[[
        'UserName', 'QueryCount', 'AvgCPUTimePerQuery', 
        'AvgElapsedTimePerQuery', 'AvgIOPerQuery'
    ]].style.format({
        'QueryCount': '{:,.0f}',
        'AvgCPUTimePerQuery': '{:.4f}',
        'AvgElapsedTimePerQuery': '{:.4f}',
        'AvgIOPerQuery': '{:,.0f}'
    }))
else:
    print("No data available for performance analysis.")

## 11. Export Results to CSV (Optional)

Save the results to CSV files for further analysis or reporting.

In [ ]:
if df is not None and not df.empty:
    # Create output directory if it doesn't exist
    output_dir = Path('output')
    output_dir.mkdir(exist_ok=True)
    
    # Generate filename with date range
    filename = f"dbql_summary_history_{start_date}_{end_date}.csv"
    output_path = output_dir / filename
    
    # Save to CSV
    df.to_csv(output_path, index=False)
    print(f"✓ Data exported to: {output_path}")
    print(f"  Rows: {len(df):,}")
    print(f"  File size: {output_path.stat().st_size / 1024**2:.2f} MB")
    
    # Also export top 10 CPU users summary
    summary_filename = f"dbql_top10_cpu_users_{start_date}_{end_date}.csv"
    summary_path = output_dir / summary_filename
    top_cpu_users.to_csv(summary_path, index=False)
    print(f"\n✓ Top 10 CPU users exported to: {summary_path}")
else:
    print("No data to export.")

## 12. Close Connections

Properly clean up database connections when done.

In [ ]:
# Close all connections
report.close()
print("✓ Database connections closed successfully!")

# Teradata DBQL Summary Table History Report

This notebook demonstrates how to retrieve DBQL summary table history from Teradata PDCR data using the `PDCRInfoReport` class.

**Report Parameters:**
- User filter: `%` (all users)
- Time range: Last 3 years
- Data source: `PDCRINFO.DBQLSummaryTbl_Hst`

**Focus:**
- Top 10 CPU users
- CPU usage trends over time

## 1. Import Required Libraries

Import necessary libraries for PDCR reporting and data analysis.

In [6]:
import logging
import sys
from pathlib import Path
from datetime import date, timedelta
import pandas as pd
import numpy as np

# Add src to path for imports
sys.path.insert(0, str(Path.cwd()))

# Import the reporting module
from src.reports import PDCRInfoReport
try:
    from src.reports import PDCRInfoReport
    from src.connection import TeradataConnectionError
except SyntaxError as e:
    print(f"✗ Syntax Error in reports.py: {e}")
    print(f"  File: {e.filename}")
    print(f"  Line: {e.lineno}")
    print(f"  Text: {e.text}")
    raise

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!


## 2. Configure Date Range

Calculate the date range for the last 3 years of data.

In [7]:
# Calculate last 3 years date range
end_date = date.today() - timedelta(days=1)  # Yesterday
start_date = end_date - timedelta(days=3*365)   # 3 years ago

# User filter pattern
user_pattern = "%"

print(f"Date Range:")
print(f"  Start Date: {start_date}")
print(f"  End Date:   {end_date}")
print(f"  User Pattern: {user_pattern}")
print(f"  Days: {(end_date - start_date).days + 1}")

Date Range:
  Start Date: 2023-01-15
  End Date:   2026-01-14
  User Pattern: %
  Days: 1096


## 3. Initialize PDCR Report Generator

Create an instance of the `PDCRInfoReport` class to access PDCR data.

In [10]:
try:
    # Initialize the report generator
    report = PDCRInfoReport()
    print("✓ PDCRInfoReport initialized successfully")
    
    # List available environments
    environments = report.conn_mgr.list_environments()
    print(f"✓ Available environments: {environments}")
    
except TeradataConnectionError as e:
    print(f"✗ Connection Error: {e}")
    print("\nPlease ensure:")
    print("1. td_env.yaml file exists in the project root")
    print("2. Copy td_env.yaml.template to td_env.yaml")
    print("3. Update credentials for your test/prod environments")

2026-01-15 21:41:23,023 - src.connection - INFO - Loaded configuration for: ['test', 'prod']


✓ PDCRInfoReport initialized successfully
✓ Available environments: ['test', 'prod']


## 4. Retrieve DBQL Summary Table History Data

Query `PDCRINFO.DBQLSummaryTbl_Hst` for all users over the last 3 years.

In [11]:
try:
    # Retrieve DBQL summary table history
    df = report.get_DBQLSummaryTable_History(
        env_name='test',  # Change to 'prod' for production data
        start_date=start_date,
        end_date=end_date,
        user_name=user_pattern
    )
    
    print(f"✓ Retrieved {len(df):,} rows from PDCRINFO.DBQLSummaryTbl_Hst")
    print(f"\nDataFrame Shape: {df.shape}")
    print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
except Exception as e:
    print(f"✗ Error retrieving DBQL summary data: {e}")
    df = None

✗ Error retrieving DBQL summary data: 'PDCRInfoReport' object has no attribute 'get_DBQLSummaryTable_History'


## 5. Display Sample Data

Preview the first few rows to understand the data structure.

In [12]:
if df is not None and not df.empty:
    print("First 10 rows:")
    display(df.head(10))
    
    print("\nColumn Data Types:")
    print(df.dtypes)
else:
    print("No data available to display.")

No data available to display.


## 6. Data Summary Statistics

Analyze the CPU usage across all retrieved data.

In [ ]:
if df is not None and not df.empty:
    print("=" * 80)
    print("DBQL SUMMARY TABLE STATISTICS")
    print("=" * 80)
    
    # Date range
    print(f"\nDate Range:")
    print(f"  First Log Date: {df['LogDate'].min()}")
    print(f"  Last Log Date:  {df['LogDate'].max()}")
    print(f"  Unique Dates:   {df['LogDate'].nunique()}")
    
    # User coverage
    print(f"\nUser Coverage:")
    print(f"  Unique Users:    {df['UserName'].nunique()}")
    print(f"  Total Queries:   {df['QueryCount'].sum():,.0f}")
    
    # CPU usage statistics
    print(f"\nAMP CPU Time (seconds):")
    print(f"  Total:   {df['AMPCPUTime'].sum():,.2f}")
    print(f"  Mean:    {df['AMPCPUTime'].mean():,.2f}")
    print(f"  Median:  {df['AMPCPUTime'].median():,.2f}")
    print(f"  Max:     {df['AMPCPUTime'].max():,.2f}")
    
    print(f"\nParser CPU Time (seconds):")
    print(f"  Total:   {df['ParserCPUTime'].sum():,.2f}")
    print(f"  Mean:    {df['ParserCPUTime'].mean():,.2f}")
    print(f"  Median:  {df['ParserCPUTime'].median():,.2f}")
    print(f"  Max:     {df['ParserCPUTime'].max():,.2f}")
    
    # Total CPU Time
    df['TotalCPUTime'] = df['AMPCPUTime'] + df['ParserCPUTime']
    print(f"\nTotal CPU Time (seconds):")
    print(f"  Total:   {df['TotalCPUTime'].sum():,.2f}")
    print(f"  Mean:    {df['TotalCPUTime'].mean():,.2f}")
    print(f"  Median:  {df['TotalCPUTime'].median():,.2f}")
    print(f"  Max:     {df['TotalCPUTime'].max():,.2f}")
    
    # I/O statistics
    print(f"\nI/O Statistics:")
    print(f"  Total I/O Count: {df['TotalIOCount'].sum():,.0f}")
    print(f"  Total I/O (GB):  {df['TotalIOInKB'].sum() / 1024**2:,.2f}")
else:
    print("No data available for analysis.")

## 7. Top 10 CPU Users

Identify the top 10 users by total CPU time consumption.

In [ ]:
if df is not None and not df.empty:
    # Calculate total CPU time per user
    if 'TotalCPUTime' not in df.columns:
        df['TotalCPUTime'] = df['AMPCPUTime'] + df['ParserCPUTime']
    
    # Aggregate by user
    user_summary = df.groupby('UserName').agg({
        'TotalCPUTime': 'sum',
        'AMPCPUTime': 'sum',
        'ParserCPUTime': 'sum',
        'QueryCount': 'sum',
        'TotalIOCount': 'sum',
        'TotalIOInKB': 'sum'
    }).reset_index()
    
    # Sort by total CPU time
    user_summary = user_summary.sort_values('TotalCPUTime', ascending=False)
    
    # Convert to hours for readability
    user_summary['TotalCPUTime_Hours'] = user_summary['TotalCPUTime'] / 3600
    user_summary['AMPCPUTime_Hours'] = user_summary['AMPCPUTime'] / 3600
    user_summary['ParserCPUTime_Hours'] = user_summary['ParserCPUTime'] / 3600
    user_summary['TotalIOInKB_GB'] = user_summary['TotalIOInKB'] / 1024**2
    
    print("\nTop 10 Users by Total CPU Time:")
    print("=" * 120)
    display(user_summary.head(10)[[
        'UserName', 'TotalCPUTime_Hours', 'AMPCPUTime_Hours', 'ParserCPUTime_Hours',
        'QueryCount', 'TotalIOInKB_GB'
    ]])
    
    # Calculate percentage of total
    total_cpu = user_summary['TotalCPUTime'].sum()
    top_10_cpu = user_summary.head(10)['TotalCPUTime'].sum()
    print(f"\nTop 10 users consume {top_10_cpu/total_cpu*100:.1f}% of total CPU time")
else:
    print("No data available for user ranking.")

## 8. Visualize Top 10 CPU Users

Bar chart showing the top 10 users by CPU consumption.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

if df is not None and not df.empty:
    # Get top 10 users
    top_users = user_summary.head(10)

    # Plot
    plt.figure(figsize=(12, 6))
    sns.barplot(
        data=top_users,
        x='UserName',
        y='TotalCPUTime_Hours',
        palette='rocket'
    )
    plt.title('Top 10 Users by Total CPU Time (Hours)', fontsize=14, fontweight='bold')
    plt.xlabel('User Name', fontsize=12)
    plt.ylabel('Total CPU Time (Hours)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 9. CPU Usage Per Day Over Period

Plot total CPU usage aggregated by day over the entire time period.

In [ ]:
import matplotlib.pyplot as plt

if df is not None and not df.empty:
    # Ensure TotalCPUTime is calculated
    if 'TotalCPUTime' not in df.columns:
        df['TotalCPUTime'] = df['AMPCPUTime'] + df['ParserCPUTime']
    
    # Aggregate by LogDate
    daily_cpu = df.groupby('LogDate')['TotalCPUTime'].sum().reset_index()
    daily_cpu['TotalCPUTime_Hours'] = daily_cpu['TotalCPUTime'] / 3600
    
    # Plot
    plt.figure(figsize=(16, 6))
    plt.plot(daily_cpu['LogDate'], daily_cpu['TotalCPUTime_Hours'], 
             marker='o', linewidth=2, markersize=4, color='#2E86AB')
    plt.title('Total CPU Usage Per Day Over Time', fontsize=14, fontweight='bold')
    plt.xlabel('Log Date', fontsize=12)
    plt.ylabel('Total CPU Time (Hours)', fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"\nDaily CPU Statistics:")
    print(f"  Average CPU per day: {daily_cpu['TotalCPUTime_Hours'].mean():,.2f} hours")
    print(f"  Peak CPU day:        {daily_cpu['TotalCPUTime_Hours'].max():,.2f} hours on {daily_cpu.loc[daily_cpu['TotalCPUTime_Hours'].idxmax(), 'LogDate']}")
    print(f"  Minimum CPU day:     {daily_cpu['TotalCPUTime_Hours'].min():,.2f} hours on {daily_cpu.loc[daily_cpu['TotalCPUTime_Hours'].idxmin(), 'LogDate']}")

## 10. CPU Usage Distribution by Top Users

Pie chart showing CPU distribution among top users.

In [ ]:
import matplotlib.pyplot as plt

if df is not None and not df.empty:
    # Get top 10 users and aggregate others
    top_n = 10
    top_users = user_summary.head(top_n)
    other_cpu = user_summary['TotalCPUTime'][top_n:].sum()
    
    # Create pie data
    pie_data = top_users[['UserName', 'TotalCPUTime']].copy()
    pie_data = pd.concat([
        pie_data,
        pd.DataFrame({'UserName': ['Others'], 'TotalCPUTime': [other_cpu]})
    ])
    
    plt.figure(figsize=(10, 8))
    plt.pie(
        pie_data['TotalCPUTime'],
        labels=pie_data['UserName'],
        autopct='%1.1f%%',
        startangle=140
    )
    plt.title('CPU Usage Distribution by User', fontsize=14, fontweight='bold')
    plt.show()

## 11. Top Users CPU Trends Over Time

Line plot showing CPU usage trends for top 6 users over time.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

if df is not None and not df.empty:
    # Get top 6 users by total CPU
    top_6_users = user_summary.head(6)['UserName'].tolist()
    
    # Filter data for top users
    df_top = df[df['UserName'].isin(top_6_users)].copy()
    
    if 'TotalCPUTime' not in df_top.columns:
        df_top['TotalCPUTime'] = df_top['AMPCPUTime'] + df_top['ParserCPUTime']
    
    # Aggregate by user and date
    user_daily = df_top.groupby(['LogDate', 'UserName'])['TotalCPUTime'].sum().reset_index()
    user_daily['TotalCPUTime_Hours'] = user_daily['TotalCPUTime'] / 3600
    
    # Plot
    plt.figure(figsize=(16, 8))
    for user in top_6_users:
        user_data = user_daily[user_daily['UserName'] == user]
        plt.plot(user_data['LogDate'], user_data['TotalCPUTime_Hours'], 
                marker='o', linewidth=2, label=user, alpha=0.7)
    
    plt.title('CPU Usage Trends for Top 6 Users Over Time', fontsize=14, fontweight='bold')
    plt.xlabel('Log Date', fontsize=12)
    plt.ylabel('Total CPU Time (Hours)', fontsize=12)
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for trend analysis.")

## 12. Query Count Analysis

Analyze query counts by user.

In [ ]:
if df is not None and not df.empty:
    # Top users by query count
    query_summary = df.groupby('UserName')['QueryCount'].sum().reset_index()
    query_summary = query_summary.sort_values('QueryCount', ascending=False)
    
    print("\nTop 20 Users by Query Count:")
    print("=" * 80)
    display(query_summary.head(20))
    
    # Plot top 10 by query count
    plt.figure(figsize=(12, 6))
    sns.barplot(
        data=query_summary.head(10),
        x='UserName',
        y='QueryCount',
        palette='viridis'
    )
    plt.title('Top 10 Users by Query Count', fontsize=14, fontweight='bold')
    plt.xlabel('User Name', fontsize=12)
    plt.ylabel('Query Count', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("No data available for query analysis.")

## 13. Export Results to CSV (Optional)

Save the results to CSV files for further analysis or reporting.

In [ ]:
if df is not None and not df.empty:
    # Create output directory if it doesn't exist
    output_dir = Path('output')
    output_dir.mkdir(exist_ok=True)
    
    # Save full data
    filename = f"dbql_summary_{start_date}_{end_date}.csv"
    output_path = output_dir / filename
    df.to_csv(output_path, index=False)
    print(f"✓ Full data exported to: {output_path}")
    print(f"  Rows: {len(df):,}")
    print(f"  File size: {output_path.stat().st_size / 1024**2:.2f} MB")
    
    # Save user summary
    user_summary_path = output_dir / f"dbql_user_summary_{start_date}_{end_date}.csv"
    user_summary.to_csv(user_summary_path, index=False)
    print(f"\n✓ User summary exported to: {user_summary_path}")
    
    # Save daily CPU data
    daily_cpu_path = output_dir / f"dbql_daily_cpu_{start_date}_{end_date}.csv"
    daily_cpu.to_csv(daily_cpu_path, index=False)
    print(f"✓ Daily CPU data exported to: {daily_cpu_path}")
else:
    print("No data to export.")

## 14. Close Connections

Properly clean up database connections when done.

In [ ]:
try:
    if 'report' in locals():
        report.close()
        print("✓ All database connections closed successfully")
    else:
        print("No report instance to close")
except Exception as e:
    print(f"✗ Error closing connections: {e}")